<a href="https://colab.research.google.com/github/ChaitaliV/Objective-criterias-to-quantify-the-accuracy-of-explanation/blob/main/Metrices/Descriptive_Accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ChaitaliV/Objective-criterias-to-quantify-the-accuracy-of-explanation

Cloning into 'Objective-criterias-to-quantify-the-accuracy-of-explanation'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 164 (delta 29), reused 0 (delta 0), pack-reused 101
Receiving objects: 100% (164/164), 8.54 MiB | 3.41 MiB/s, done.
Resolving deltas: 100% (73/73), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install shap
!pip install lime
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 KB 10.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 KB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283859 sha256=91bea2433ec764671e9bd4fe69b6c321f8c727f65ece32fc93eff28e43c303fa
  Stored in directory: /root/.cache/pip/wheels/ed/d7/c9/5a0130d06d6310bc6cbe55220e6e72dcb8c4eff9a478717066
Successfully built lime
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 26.

In [4]:
import torch
import pandas as pd
import numpy as np
import shap
from transformers import BertTokenizer
import scipy as sp
import torch
import lime
import tensorflow as tf
from lime.lime_text import LimeTextExplainer
import transformers
import torch.nn as nn

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
model=torch.load(r'/content/drive/MyDrive/Saved Models/MuRIL.pt',map_location=torch.device('cuda'))
tokenizer = BertTokenizer.from_pretrained('google/muril-base-cased', do_lower_case=True)    

In [7]:
df = pd.read_csv('/content/Objective-criterias-to-quantify-the-accuracy-of-explanation/Dataset/test .csv')
df.head()

,hindi_text,label
0,मैं न्यूयॉर्क में अपने दादा-दादी के साथ पैदा ह...,0
1,मैं किशोरावस्था से ही गंभीर अवसाद के दौर से गु...,1
2,मुझे शौक के तौर पर खाना बनाना पसंद है,0
3,"मैं इन दिनों बहुत कम महसूस कर रहा हूं, ऐसा महस...",1
4,हाल ही में ब्रिटेन की महारानी एलिजाबेथ का निधन...,2


In [8]:
text = df['hindi_text'][0]

In [9]:
test_data = df['hindi_text'][:10]
test_label = df['label'][:10]

In [10]:
# Define a function to preprocess the text data for the MuRIL model

def adapter(data):
  #inputs = tokenizer(data, return_tensors="pt")
  inputs = tokenizer(data, 
          return_tensors='pt', 
          padding=True, 
          truncation=True, 
          max_length=128)

  with torch.no_grad():
      #m = model(**inputs)
      #print(model(**inputs).logits)
      logits = model(**inputs).logits
      scores = model(**inputs)[0].softmax(1).detach().cpu().numpy()
      # predictions = tf.nn.softmax(logits)
      # prediction_label = class_name[np.argmax(predictions[0])]
  
  return scores


In [11]:
## generate explanation with LIME

class_name = ['Introduction','Depression','Grey Area']
explainer = LimeTextExplainer(class_names = class_name)

lime_explanations = []
for i in range(len(test_data)):
    text = test_data.iloc[i]
    label = test_label.iloc[i]
    explanation = explainer.explain_instance(text,adapter)
    exp = explanation.as_list()
    lime_explanations.append(exp)

In [12]:
## SHAP adaptor and predictor
def adapter_shap(data):
  #inputs = tokenizer(data, return_tensors="pt")
  inputs = tokenizer(data, 
          return_tensors='pt', 
          padding=True, 
          truncation=True, 
          max_length=128)

  with torch.no_grad():
      logits = model(**inputs).logits
      scores = model(**inputs)[0].softmax(1).detach().cpu().numpy()

  predicted_class_id = logits.argmax().item()
  classifications = ['Introduction','Depression','Grey Area']
  classification = logits.argmax().item()
  class_name = classifications[classification]
  return class_name

def predictor(x):
    probas = model(**tokenizer(x, return_tensors="pt", padding=True))[0].softmax(1).detach().cpu().numpy()
    # logits = model(**tokenizer(x, return_tensors="pt", padding=True)).logits
    # predicted_class_id = logits.argmax().item()
    # classifications = ['Introduction','Depression','Grey Area']
    # classification = logits.argmax().item()
    # class_name = classifications[classification]
    # print(class_name)
    s = sp.special.logit(probas[:,1])
    return s

def f_batch(x):
    val = np.array([])
    for i in x:
      val = np.append(val, predictor(i))
    return val

In [13]:
explainer = shap.Explainer(f_batch, tokenizer)
def explain(data):
  test = { 'text': [data]}
  shap_values = explainer(test)
  values = shap_values[0].values
  return values


In [14]:
# Generate SHAP explanations for the test data
shap_explanations = []
for i in range(len(test_data)):
    text = test_data.iloc[i]
    exp = explain(text)
    shap_explanations.append(exp)

  0%|          | 0/498 [00:00<?, ?it/s]

In [22]:
lime_scores = []
for instance in lime_explanations:
  individual_score = []
  for tuple in instance:
    feature, score = tuple
    individual_score.append(score)
  lime_scores.append(individual_score)

In [40]:
shap_scores = []
for instance in shap_explanations:
  individual_score = instance.tolist()
  shap_scores.append(individual_score)

In [105]:
def descriptive_accuracy_lime(lime_scores, threshold):
    """
    Computes the descriptive accuracy of LIME explanations given a threshold.
    """
    num_instances = len(lime_scores)
    lime_correct = np.sum(np.abs(lime_scores) >= threshold)
    lime_coverage = lime_correct / num_instances
    lime_acc = lime_correct / np.sum(np.abs(lime_scores))
    
    return lime_acc, lime_coverage

In [106]:
# Assuming that you have the LIME scores in a numpy array, lime_scores
threshold = 0.1
lime_acc, lime_coverage = descriptive_accuracy_lime(lime_scores, threshold)
print(f"LIME accuracy: {lime_acc:.3f}, LIME coverage: {lime_coverage:.3f}")

LIME accuracy: 3.364, LIME coverage: 1.800


In [110]:
def descriptive_accuracy_shap(shap_scores, threshold):
    """
    Computes the descriptive accuracy of LIME and SHAP explanations given a threshold.
    """
    max_features = max([len(score) for score in shap_scores])
    num_instances = len(shap_scores)
    # Pad the SHAP scores with zeros to make them equal in length
    shap_scores_padded = []
    for score in shap_scores:
        if len(score) < max_features:
            score = np.pad(score, (0, max_features - len(score)), mode='constant')
        shap_scores_padded.append(score)
    
    # Convert the padded SHAP scores to a numpy array
    shap_scores_padded = np.array(shap_scores_padded)
    shap_correct = np.sum(np.absolute(shap_scores_padded) >= threshold)
    shap_coverage = shap_correct / num_instances
    shap_acc = shap_correct / np.sum(np.abs(shap_scores_padded))
  
    return shap_acc, shap_coverage

In [111]:
# Assuming that you have the SHAP scores in a numpy array, lime_scores
threshold = 0.1
shap_acc, shap_coverage = descriptive_accuracy_shap(shap_explanations,threshold)
print(f"SHAP accuracy: {shap_acc:.3f}, SHAP coverage: {shap_coverage:.3f}")

SHAP accuracy: 1.880, SHAP coverage: 5.300
